In [ ]:
#Import dependencies
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time
import requests
from bs4 import BeautifulSoup
import re
import config
import json
import os
from urllib.parse import urlparse
import csv

In [ ]:
#specify the path to chromedriver.exe (download and save on your computer)
driver = webdriver.Chrome()

#open the webpage
driver.get("https://www.instagram.com/")

In [ ]:
#target username
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

#enter username and password
username.clear()
username.send_keys(config.username)
password.clear()
password.send_keys(config.password)

#target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

In [ ]:
# not_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Not Now")]')))

# not_button.click()

In [ ]:
# Wait up to 10 seconds for the search button to be clickable on the web page
search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'svg[aria-label="Search"]')))

# Click the search button once it becomes clickable
search_button.click()

In [ ]:
#target the search input field
searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Search']")))
searchbox.clear()

#search for the @handle or keyword (loop function will come here to loop through profiles)
keyword = "@username" # this needs to be replacd with a loop to open profiles step by step
searchbox.send_keys(keyword)

In [ ]:
# Check if the keyword starts with "@"
if keyword.startswith("@"):
    # Removes the "@" symbol
    keyword = keyword[1:]
    
try:
    # Wait until the element with the specified XPath and text is present and clickable
    first_result = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f'//span[text()="{keyword}"]'))
    )
    print(first_result)
    
    # Click on the found element
    first_result.click()
    print(f"Clicked on the element for keyword: {keyword}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# Get the initial page height
initial_height = driver.execute_script("return document.body.scrollHeight")

# Create a list to store htmls
soups = []

while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait for a moment to allow new content to load (adjust as needed)
    time.sleep(10)
    
    # Parse the HTML
    html = driver.page_source
    
    # Create a BeautifulSoup object from the scraped HTML
    soups.append(BeautifulSoup(html, 'html.parser'))

    # Get the current page height
    current_height = driver.execute_script("return document.body.scrollHeight")

    if current_height == initial_height:
        break  # Exit the loop when you can't scroll further
 
    initial_height = current_height  # Update the initial height for the next iteration

In [ ]:
# List to store the post image URLs
post_urls = []

for soup in soups:
    # Find all anchor elements with href attributes
    anchors = soup.find_all('a', href=True)
    # print(anchors[9])
    # print([anchor['href'] for anchor in anchors])  # Prints only the href attribute of each <a> tag
    # print([anchor['href'] for anchor in anchors if anchor['href'].startswith(("/p/", "/reel/"))])

    # Filter URLs that start with "/p/" or "/reel/"
    post_urls.extend([anchor['href'] for anchor in anchors if anchor['href'].startswith((f"/{keyword}/reel/",f"/{keyword}/p/"))])
    # print(post_urls[9])

# Convert the list to a set to remove duplicates
unique_post_urls = list(set(post_urls))

print(f"before: {len(post_urls)}, after: {len(unique_post_urls)}")

In [ ]:
for items in unique_post_urls:
    print(items)

In [ ]:
!pip install instaloader

In [ ]:
# import instaloader

# # Initialize Instaloader
# loader = instaloader.Instaloader()

# # URL of the Instagram post
# post_url = "https://www.instagram.com/p/POST_ID/"  # Replace POST_ID with actual post ID or full URL

# # Download the post
# try:
#     loader.download_post(instaloader.Post.from_shortcode(loader.context, "POST_ID"), target="downloaded_content")
#     print("Video downloaded successfully!")
# except Exception as e:
#     print("An error occurred:", e)


In [ ]:
import re

# List to store extracted post shortcodes
unique_post_shortcodes = []

# # Loop through each URL and extract the post ID
for text in unique_post_urls:
    match = re.search(r'/(?:p|reel)/([^/]+)/', text)
    if match:
        post_id = match.group(1)
        unique_post_shortcodes.append(post_id)

# Print the result
print(unique_post_shortcodes)
print("\n Total unique post shortcodes: " + str(len(unique_post_shortcodes)))

In [ ]:
# This is the best code (lacks replies extraction)
# Extract caption, hashtags, and comments
import instaloader
import os

# Initialize Instaloader
loader = instaloader.Instaloader()

# Login to Instagram using instaloader
# instagram login and instaloader login operate independently
username = config.username  # Replace with your Instagram username
password = config.password  # Replace with your Instagram password
loader.login(username, password)

# Create a folder for the posts 
target_folder = "downloaded_content"
os.makedirs(target_folder, exist_ok=True)

# Loop through each post ID
for post_shortcode in unique_post_shortcodes:
    try:
        # Load the post
        post = instaloader.Post.from_shortcode(loader.context, post_shortcode)

        # Download media content for the post
        # loader.download_post(post, target=target_folder)

        if post.caption:
            caption = post.caption 
        else:
            caption = "No Captions"
        
        if post.caption_hashtags:
            hashtags = post.caption_hashtags
        else:
            hashtags = []
        
        # Handle comments as a list
        comments = []
        for comment in post.get_comments():
            comments.append({
                "user": comment.owner.username,
                "text": comment.text,
                "created_at": comment.created_at_utc
            })
    
        # Define a unique metadata file for each post
        metadata_file_path = os.path.join(target_folder, f"metadata.txt")
        
        # Save caption, hashtags, and comments to a file in the target folder
        with open(metadata_file_path, "a", encoding="utf-8") as file:
            file.write("Caption:\n")
            file.write(caption + "\n\n")
        
            file.write("Hashtags:\n")
            if hashtags:
                file.write(" ".join(f"#{tag}" for tag in hashtags) + "\n\n")
            else:
                file.write("  No Hashtags" + "\n\n")
        
            file.write("Comments:\n")
            if comments:
                for comment in comments:
                    file.write(f"{comment['user']} ({comment['created_at']}): {comment['text']}\n\n")
            else:
                file.write("  No Comments\n\n")
        
        print(f"Downloaded post and metadata for {post_shortcode} successfully!")
    
    except Exception as e:
        print(f"An error occurred with post {post_shortcode}: {e}")

In [ ]:
# trail code to extract replies from comment 
# delete if it does not extract replies from comments

import instaloader

# Initialize Instaloader
loader = instaloader.Instaloader()

# Login to Instagram using instaloader
username = config.username  # Replace with your Instagram username
password = config.password  # Replace with your Instagram password
loader.login(username, password)

# Define the post shortcode you want to analyze
post_shortcode = "CiY9_qaPYqD"  # Replace with the Instagram post shortcode

# Load the post
post = instaloader.Post.from_shortcode(loader.context, post_shortcode)
post = instaloader.Post.from_shortcode(loader.context, post_shortcode)

# List to hold comments and replies
comments_data = []

# Loop through each comment in the post
for comment in post.get_comments():
    comment_info = {
        "user": comment.owner.username,
        "text": comment.text,
        "created_at": comment.created_at_utc,
        "replies": []
    }

    # Check if there are replies to the comment
    for reply in comment.answers:
        reply_info = {
            "user": reply.owner.username,
            "text": reply.text,
            "created_at": reply.created_at_utc
        }
        comment_info["replies"].append(reply_info)
    
    comments_data.append(comment_info)

# Print comments and their replies
for comment in comments_data:
    print(f"Comment by {comment['user']} ({comment['created_at']}): {comment['text']}")
    for reply in comment["replies"]:
        print(f"  Reply from {reply['user']} ({reply['created_at']}): {reply['text']}")